In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/Defi-PassAI

In [7]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import unicodedata
import string
import random
from torch.utils.data import Dataset, DataLoader

In [9]:
# check which device we gonna use "GPU" or "CPU"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
#get all the letters on training and evaluation datasets
with open("data/train.txt","r") as f:
  txt = f.read()
 
all_letters = sorted(list(set(txt)))
all_letters += ['<EOS>']
n_letters = len(all_letters)

In [ ]:
print(n_letters)

94


In [ ]:
# a function that return the index of a character
def letterToIndex(letter):
    if letter in all_letters:
      return all_letters.index(letter)
    return 0

In [ ]:
# Read a file and split into lines
def readLines(filename):
    lines = open(filename).read().strip().split('\n')
    return [line.strip('\\') for line in lines] # remove the \ caracters from the end of passwords in the eval dataset

In [ ]:
lines = readLines("data/train.txt")
max_length = max([len(word) for word in lines]) #get the maximum lenght of a password in the training set
print(max_length)

110


In [ ]:
# create a dataset of classe that we will use to train the model 
class PasswordsDataset(Dataset):
    
    def __init__(self, passwords_list):
        self.passwords_list = passwords_list
        
    def __len__(self):
        return len(self.passwords_list)
    
    # it will return an X Y fo each line of the dataset X will be the table of onehot encoded array describing each character of the line [0:n]
    # and Y will be the line from [1:n] + the end of word '\\'
    # example: x= 'abcd' y='bcd\\'
    def __getitem__(self, idx):
        x_str = self.passwords_list[idx].ljust(max_length, '\\')[:max_length]
        y_str = x_str[1:] + '\\'
        x = torch.zeros((max_length, n_letters))
        y = torch.zeros(max_length)
        for i, c in enumerate(x_str):
            x[i, letterToIndex(c)] = 1
        for i, c in enumerate(y_str):
            y[i] = letterToIndex(c)
            
        return x, y

In [ ]:
trainset = PasswordsDataset(lines)

In [ ]:
train_loader = DataLoader(trainset, batch_size=256, shuffle=True)
print(len(train_loader))

1469


In [ ]:
train_iter = train_loader.__iter__()
X, Y = train_iter.next()
print(X.size(), Y.size())

torch.Size([256, 110, 94]) torch.Size([256, 110])


In [ ]:
# hyper parameters
input_size = n_letters
hidden_size = 54
output_size = n_letters
num_layers = 1
train_batch_size = 256

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc3 = nn.Linear(output_size, output_size)
        
    def forward(self, X, states):
        ht, ct = states
        batch_size = X.size(0)
        out, (ht, ct) = self.lstm1(X, (ht, ct))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out, (ht, ct) # out: Size([batch_size, max_length, num_chars])

In [ ]:
model = Model(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers)
model = nn.DataParallel(model)
model = model.to(device)

In [ ]:
ht = torch.zeros((num_layers, train_batch_size, hidden_size)).to(device)
ct = torch.zeros((num_layers, train_batch_size, hidden_size)).to(device)

In [ ]:
lr = 0.005
step_size = len(train_loader) * 1
gamma = 0.95
print(step_size)

1469


In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=step_size, gamma=gamma)

In [ ]:
def generate_password(model, start='a', k=5):
    
    if len(start) >= max_length:
        return password
    
    with torch.no_grad():
        
        ht = torch.zeros((num_layers, 1, hidden_size)).to(device)
        ct = torch.zeros((num_layers, 1, hidden_size)).to(device)
        length = 0
        password = start
        # model preparation to predict the rest of the password based on `start`
        for char in start:
            X = torch.zeros((1, 1, n_letters)) # [batch_size, timestep, n_letters]
            X[0, 0, letterToIndex(char)] = 1
            out, (ht, ct) = model(X, (ht, ct))
            length += 1
        vals, idxs = torch.topk(out[0], k) # 0 -> first eg in a batch
        idx = np.random.choice(idxs.cpu().numpy()[0]) # 0 -> first...
        char = all_letters[idx]
        vals, idxs = torch.topk(out[0], k) # 0 -> first eg in a batch
        idx = np.random.choice(idxs.cpu().numpy()[0]) # 0 -> first...
        char = all_letters[idx]
        
        while char != "\\" and length <= max_length-1:
            X = torch.zeros((1, 1, n_letters)) # [batch_size, timestep, n_letters]
            X[0, 0, letterToIndex(char)] = 1
            out, (ht, ct) = model(X, (ht, ct))
            vals, idxs = torch.topk(out[0], k) # 0 -> first eg in a batch
            idx = np.random.choice(idxs.cpu().numpy()[0]) # 0 -> first...
            char = all_letters[idx]
            length += 1
            password += char
    
        if password[-1] != "\\":
            password += "\\"
    
    return password

In [ ]:
passwords_list = lines
def sampler(model, start='a', n=10, k=5, only_new=False):
    
    passwords = []
    cnt = 0
    while cnt <= n:
        password = generate_password(model=model, start=start, k=k)
        if only_new: 
            if password not in passwords_list and password not in passwords:
                passwords.append(password)
                cnt += 1
        else:
            if password not in passwords:
                passwords.append(password)
                cnt += 1
    passwords = [password[:-1] for password in passwords]
    
    return passwords

In [ ]:
epochs = 20

# if you are testing this do not run the next cell 
# this is only for training it gonna take hours to end 

In [ ]:
from tqdm.notebook import tqdm

epoch_losses = []
epoch_lrs = []
iteration_losses = []
iteration_lrs = []

for epoch in tqdm(range(1, epochs+1), desc="Epochs"):
    epoch_loss = 0
    epoch_lr = 0
    
    for i, (X, Y) in tqdm(enumerate(train_loader, 1), total=len(train_loader), desc="Epoch-{}".format(epoch)):
    #for i, (X, Y) in enumerate(train_loader, 1):
        X, Y = X.to(device), Y.to(device)
        
        ht = torch.zeros((num_layers, X.size(0), hidden_size)).to(device)
        ct = torch.zeros((num_layers, X.size(0), hidden_size)).to(device)

        optimizer.zero_grad()
        Y_pred_logits, (ht, ct) = model(X, (ht, ct))
        Y_pred_logits = Y_pred_logits.transpose(1, 2) # Check Loss Doc: [N, d1, C] -> [N, C, d1]
        loss = criterion(Y_pred_logits, Y.long())
        loss.backward(retain_graph=True)
        optimizer.step()
        lr_scheduler.step()
        
        iteration_losses.append(loss.item())
        iteration_lrs.append(lr_scheduler.get_last_lr()[0])
        epoch_loss += loss.item()
        epoch_lr += lr_scheduler.get_last_lr()[0]
        
    epoch_loss /= len(train_loader)
    epoch_lr /= len(train_loader)
    epoch_losses.append(epoch_loss)
    epoch_lrs.append(epoch_lr)

    message = "Epoch:{}    Loss:{}    LR:{}".format(epoch, epoch_loss, epoch_lr)
    print(message)
    passwords = sampler(model, start='jo', n=10, k=10, only_new=False)
    print(passwords)
    torch.save(model.state_dict(), "password_gen_" + str(epoch) + ".pth")

Epochs:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch-1:   0%|          | 0/1469 [00:00<?, ?it/s]

Epoch:1    Loss:0.22215488488127214    LR:0.004999829816201406
['joitses71480', 'joimia1170516948050388', 'josondangonsenksan134', 'jospe158100225', 'jottss26', 'joigith002056643694806869638647255467577178489534', 'jogy', 'jootatey686451980281157', 'joeystacaraledatayduk1065063347630363640148840276761275000370660', 'jogoongut17123002036913137705', 'jontar']


Epoch-2:   0%|          | 0/1469 [00:00<?, ?it/s]

Epoch:2    Loss:0.18103079991814552    LR:0.004749838325391262
['joeliti', 'jomirlotot12', 'jooti', 'joa16970162147679559981511352375423457865', 'joesor26835257854280854407', 'jorloge674379914968823538288789328745477438737101410868396787', 'josom3', 'jodybey', 'joct109', 'joceta', 'josotan14415572168852']


...

Epoch:18    Loss:0.1679888560282608    LR:0.002090530518789069
['joad12320694', 'jo156', 'jodworm4', 'joar', 'joos13757024842', 'jojdjjj11', 'josacoca', 'joel1259689005', 'jonty151533652086', 'joetty111374562721914273', 'joj157867792']


Epoch-19:   0%|          | 0/1469 [00:00<?, ?it/s]

Epoch:19    Loss:0.16785568682574512    LR:0.0019860039928495912
['jomessoun2', 'joons285', 'jobay', 'jos29396056563130017101', 'jous', 'joan02342637651001729697886281976062', 'jok255', 'josurerianieeer0061188662867322', 'jogiansa051419', 'jojy', 'joakk1838963816005']


Epoch-20:   0%|          | 0/1469 [00:00<?, ?it/s]

Epoch:20    Loss:0.16773943406250139    LR:0.0018867037932070636
['joact', 'joim', 'joikkk', 'jouou2', 'jopita262', 'joa108057999', 'josokara1', 'jof39058055', 'josm19339884012876243719366', 'jodric1512002815847190052274622443', 'jojittass20346']


In [ ]:
ls 

 1M.txt                          password_gen_13.pth   password_gen_2.pth
'Copy of 1M.txt'                 password_gen_14.pth   password_gen_3.pth
 data/                           password_gen_15.pth   password_gen_4.pth
 Defi-SecAIDecembre.ipynb        password_gen_16.pth   password_gen_5.pth
'LSTM Password Genrator.ipynb'   password_gen_17.pth   password_gen_6.pth
 model/                          password_gen_18.pth   password_gen_7.pth
 password_gen_10.pth             password_gen_19.pth   password_gen_8.pth
 password_gen_11.pth             password_gen_1.pth    password_gen_9.pth
 password_gen_12.pth             password_gen_20.pth   tests/


In [ ]:
# we load the prevouis trained model
path = "password_gen_20.pth" # change this name if you retrained the model 
model = Model(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers)
model = nn.DataParallel(model)
model = model.to(device)
model.load_state_dict(torch.load(path,map_location=torch.device('cpu'))) 
# model.load_state_dict(torch.load(path)) 

<All keys matched successfully>

Generating 10k password

In [ ]:
rm 10k.txt

In [ ]:
# x = sampler(model,start="m", n=10,k=5, only_new=True)
# xy = []
# for i in x:
#   print(type(i+'\\'))
#   xy.append(i+'\\')

# xx = [i[:-1] for i in x]
# print(x) 
# print(xx) 
# print(xy)
# print(x)
# with open("10k.txt", "w") as f:
#   for i in x:
#     f.write(i)
#     # print(type(i))

# with open("10k.txt") as f:
#   print("reading file")
#   f.read()
# generating 1M password
start_letter = all_letters
with open('1M.txt','a') as f:
  for i in range(600000):
    r_letter = random.randrange(0,len(start_letter))
    res = sampler(model, start=start_letter[r_letter], n=0, k=5, only_new=True)
    f.write(res[0]+"\\\n")

In [ ]:
ls

 1M.txt                          password_gen_13.pth   password_gen_2.pth
'Copy of 1M.txt'                 password_gen_14.pth   password_gen_3.pth
 data/                           password_gen_15.pth   password_gen_4.pth
 Defi-SecAIDecembre.ipynb        password_gen_16.pth   password_gen_5.pth
'LSTM Password Genrator.ipynb'   password_gen_17.pth   password_gen_6.pth
 model/                          password_gen_18.pth   password_gen_7.pth
 password_gen_10.pth             password_gen_19.pth   password_gen_8.pth
 password_gen_11.pth             password_gen_1.pth    password_gen_9.pth
 password_gen_12.pth             password_gen_20.pth   tests/


In [ ]:
open('data/eval.txt','r').readlines()

In [ ]:
"""
this function will test the model over the data in eval.txt
we will take every password and pass its first letters and then we will see if 
the model predict the password , and we will fdo that the first time by giving it just 1 letter and the secon 2 letters and sow on till 5 letters
"""
from tqdm.notebook import tqdm

def test():
  test_passwords = []
  results = [0,0,0,0,0]
  with open("data/eval.txt") as f:
    test_passwords = f.readlines()
  for password in tqdm(test_passwords):
    for i in range(1,5): # to take 1 letter than 2 that 3 ..... till 5
      password_guesses = sampler(model, start=password[:i], n=10, k=5 , only_new=True)
      # found = []
      if password+"\\" in password_guesses:
        results[i-1]+=1
        # found.append(password)
  for i in range(5):
    results[i]/=len(test_passwords)
  return results

In [ ]:
res = test()


NameError: ignored

In [ ]:
print("if we give the model: ")
for i,j in enumerate(res):
  print("\t",i,"lettes it predict:",j*100,"% password correct.")

if we give the model: 
	 0 lettes it predict: 0.0 % password correct.
	 1 lettes it predict: 0.05002501250625312 % password correct.
	 2 lettes it predict: 0.0 % password correct.
	 3 lettes it predict: 0.05002501250625312 % password correct.
	 4 lettes it predict: 0.0 % password correct.
